In [1]:
import numpy as np
import pandas as pd
#import fiona 
import geopandas as gpd
#import math
#from netCDF4 import Dataset
import xarray as xa
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [2]:
# File written by dday_slope-radadj.ipynb
path = 'c:/Users/markstro/work1.1/weather/'
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'

#fns = [("2011", "2011_gm_climate_2020_03_02.nc"), ("2012", "2012_gm_climate_2020_03_03.nc"),
#       ("2013", "2013_gm_climate_2020_03_05.nc"), ("2014", "2014_gm_climate_2020_03_06.nc"),
#       ("2015", "2015_gm_climate_2020_03_08.nc"), ("2016", "2016_gm_climate_2020_03_09.nc"),
#       ("2017", "2017_gm_climate_2020_03_10.nc"), ("2018", "2018_gm_climate_2020_03_12.nc")]

fns = [("2012", "2012_gm_climate_2020_03_03.nc")]

param_path = "c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_master/"

In [3]:
prcp = xa.Dataset()
tmax = xa.Dataset()
time = xa.Dataset()

In [5]:
for fn in fns:
    yr_str = fn[0]
    ds = xa.open_dataset(path + fn[1])

    tmax[yr_str] = ds["tmax"].load()
    prcp[yr_str] = ds["prcp"].load()
    time[yr_str] = ds["time"].load()

In [6]:
for fn in fns:
    yr_str = fn[0]
    print(tmax[yr_str])

<xarray.DataArray '2012' (time: 366, hruid: 139807)>
array([[ 15.05872   ,  15.006667  ,  14.772855  , ...,  -1.6914458 ,
          0.86016136,  -2.0596802 ],
       [ 12.877452  ,  12.806542  ,  12.6142645 , ...,   2.960755  ,
          9.464185  , -12.147482  ],
       [ 13.137869  ,  13.099908  ,  12.931471  , ...,   0.76221454,
         10.1491375 ,  -7.036676  ],
       ...,
       [ 11.307938  ,  11.178908  ,  11.0968275 , ...,  -6.8468275 ,
         -1.3102446 ,  -5.5982814 ],
       [ 11.688894  ,  11.672145  ,  11.683639  , ...,  -7.709694  ,
         -0.99512464,  -5.53213   ],
       [ 17.374388  ,  17.434303  ,  17.19862   , ...,  -9.528326  ,
         -2.0833645 ,  -6.5600867 ]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-12-31
  * hruid    (hruid) int32 76128 76148 76171 76171 ... 64081 64151 65634 18844
Attributes:
    long_name:      Maximum daily air temperature
    units:          degree_Celsius
    standard_name:  ma

In [7]:
for fn in fns:
    yr_str = fn[0]
    print(prcp[yr_str])

<xarray.DataArray '2012' (time: 366, hruid: 139807)>
array([[ 0.        ,  0.        ,  0.        , ...,  1.483316  ,
         0.        ,  1.4881951 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.3511276 ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.9200325 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.03394662,
         0.        ,  0.30924857],
       [10.474995  ,  9.810726  , 11.73361   , ...,  0.0238622 ,
         0.        ,  0.        ]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-12-31
  * hruid    (hruid) int32 76128 76148 76171 76171 ... 64081 64151 65634 18844
Attributes:
    long_name:      Daily precipitation rate
    units:          mm/day
    standard_name:  lwe_precipitation_rate


In [8]:
for fn in fns:
    yr_str = fn[0]
    print(time[yr_str])

<xarray.DataArray '2012' (time: 366)>
array(['2012-01-01T00:00:00.000000000', '2012-01-02T00:00:00.000000000',
       '2012-01-03T00:00:00.000000000', ..., '2012-12-29T00:00:00.000000000',
       '2012-12-30T00:00:00.000000000', '2012-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01 2012-01-02 ... 2012-12-31
Attributes:
    long_name:      time
    standard_name:  time


In [ ]:
nhru_v11 = gpd.read_file(gdb_path, layer='nhru_v11')

In [ ]:
nhru_v11_ids = nhru_v11["nhru_v11"].values
nhru = len(nhru_v11_ids)
print(nhru)
print(nhru_v11_ids)

nmonths = 12
print(nhru * nmonths)

In [ ]:
# Read PRMS parameters
tmax_allrain_offset_f = pd.read_csv(param_path + "tmax_allrain_offset.csv")
tmax_allrain_offset = tmax_allrain_offset_f["tmax_allrain_offset"].values

tmax_allsnow_f = pd.read_csv(param_path + "tmax_allsnow.csv")
tmax_allsnow = tmax_allsnow_f["tmax_allsnow"].values
tmax_allrain = tmax_allsnow + tmax_allrain_offset

print(tmax_allrain)
print(tmax_allrain.shape)

tmax_allrain.shape = (nmonths, nhru)

print(tmax_allrain.shape)

# These 2D parameters are all set to constants
ppt_rad_adj = 0.02
tmax_index = 50.0
radj_sppt = 0.44
radj_wppt = 0.55
radadj_intcp = 1.0
radadj_slope = 0.02

In [ ]:
nhru_v11.head()

In [ ]:
#! Set precipitation adjument factor based on temperature
#! and amount of precipitation
# pptadj = 1.0
# IF ( Hru_ppt(j)>Ppt_rad_adj(j,Nowmonth) ) THEN
# IF ( Tmax_hru(j)<Tmax_index(j,Nowmonth) ) THEN
#   pptadj = Radj_sppt(j)
#   IF ( Tmax_hru(j)>=Tmax_allrain(j,Nowmonth) ) THEN
#     IF ( Summer_flag==0 ) pptadj = Radj_wppt(j) ! Winter
#   ELSE
#     pptadj = Radj_wppt(j)
#   ENDIF
# ELSE
#   pptadj = Radadj_intcp(j, Nowmonth) + &
# &                 Radadj_slope(j, Nowmonth)*(Tmax_hru(j)-Tmax_index(j,Nowmonth))
#   IF ( pptadj>1.0 ) pptadj = 1.0
# ENDIF
# ENDIF
        
for fn in fns:
    yr_str = fn[0]
    time_da = time[yr_str]
    time_vals = time_da.values
    for itime in time_vals:
        print(itime)